<a href="https://colab.research.google.com/github/hrushik98/text2Sql-llama/blob/main/Llama_v2_13b_text2SQL_finetuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
huggingface_auth_token = "<auth_key_here>" #get yours from: https://huggingface.co/meta-llama/Llama-2-13b-chat-hf

In [1]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb
!pip install gdown -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 892.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9

download the finetuned weights from drive ⬇

In [2]:
!mkdir outputs
!gdown https://drive.google.com/file/d/19TRNTEYyMBcA9JuAOPW8cogVah2CJzpq/view?usp=sharingg -O "outputs/adapter_model.bin"


/usr/local/lib/python3.10/dist-packages/gdown/parse_url.py:35: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=19TRNTEYyMBcA9JuAOPW8cogVah2CJzpq
  warnings.warn(
Downloading...
From: https://drive.google.com/file/d/19TRNTEYyMBcA9JuAOPW8cogVah2CJzpq/view?usp=sharingg
To: /content/outputs/adapter_model.bin
79.5kB [00:00, 3.60MB/s]


In [3]:
import json

# Define the content as a Python dictionary
adapter_config = {
    "auto_mapping": None,
    "base_model_name_or_path": "meta-llama/Llama-2-13b-hf",
    "bias": "none",
    "fan_in_fan_out": False,
    "inference_mode": True,
    "init_lora_weights": True,
    "layers_pattern": None,
    "layers_to_transform": None,
    "lora_alpha": 16,
    "lora_dropout": 0.1,
    "modules_to_save": None,
    "peft_type": "LORA",
    "r": 64,
    "revision": None,
    "target_modules": ["q_proj", "v_proj"],
    "task_type": "CAUSAL_LM"
}



# Save the dictionary as a JSON file
file_path = "/content/outputs/adapter_config.json"  # Change the path as needed
with open(file_path, "w") as json_file:
    json.dump(adapter_config, json_file, indent=4)

print(f"The adapter_config.json file has been created in the Colab environment.")


The adapter_config.json file has been created in the Colab environment.


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "meta-llama/Llama-2-13b-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    use_auth_token = f"{huggingface_auth_token}",
)
model.config.use_cache = False

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:460: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_auth_token = f"{huggingface_auth_token}")
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from peft import LoraConfig, get_peft_model

load the weights and the config files ⬇

In [ ]:
lora_config = LoraConfig.from_pretrained('outputs/')
model = get_peft_model(model, lora_config)

start query ⬇

In [ ]:
!pip install pandas openpyxl
import pandas as pd

In [ ]:
input = """

  You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

  You must output the SQL query that answers the question.

  ### Input:
  Count the average Wins which has a Class of 250cc, and a Year of 1972?

  ### Context:
  CREATE TABLE table_name_1 (wins INTEGER, class VARCHAR, year VARCHAR)

  ### Response:
"""

In [ ]:
device = "cuda:0"


inputs = tokenizer(input, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

